# Imports

In [1]:
# Python packages
import sys
sys.path.append('../../')
from datetime import datetime
import numpy as np
import pandas as pd
import pickle

# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV, cross_validate, KFold, cross_val_score
# from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, LassoCV
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer, make_column_transformer
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.svm import SVR
# from sklearn.impute import SimpleImputer
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.decomposition import PCA
# from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRegressor, XGBClassifier

# Custom functions
import src.settings as settings
import src.mapper_cols as mapper_cols
from src.run_all.get_data import get_data
from src.run_all.preprocess import preprocess_data
from src.run_all.train import train_and_fit_models
from src.utilities.utilities import get_latest_file, list_filenames

# instellingen voor panda weergave aanpassen
pd.set_option('display.max_rows', 500) # alle rijen tonen
pd.set_option('display.max_columns', 500) # alle kolommen tonen
pd.set_option('display.width', 1000) # kolombreedte
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.15f}'.format(x)) # floats output tot 15 decimalen

### Settings

In [2]:
# Location all data
# datapath = '../../data/'
# # Drop target values in X set
# X_DROP_VALUES = settings.Y_TARGET_COLS
# # de kolom die wordt gebruikt als y value
# Y_VALUE = ['wmoclientenper1000inwoners']
# # test size voor de train/test split
# TEST_SIZE = 0.3
# # random state voor de train/test split. Bijvoorbeeld random_state = 42 als vaste seed voor reproduceerbaarheid
# RANDOM_STATE = 42

# ## Model parameters
# # manier van cross validate in de modellen. Bijvoorbeeld 10 of RepeatedKFold(n_splits=30, n_repeats=5, random_state=1)
# CROSS_VALIDATE = 5
# # manier van scoren in de modellen
# MODEL_SCORING = 'neg_mean_squared_error'
# ## Grid Search parameters

# # parameters die gebruikt worden in de grid search
# ALPHA = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
# NEIGHBORS = [3, 5, 11, 19]
# NORMALIZE = [True, False]
# KERNEL = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
# GAMMA = [0.5, 1, 1.5, 2, 5]
# N_ESTIMATORS = [50,100,200]
# C_REGULARIZATION = [0.001, 0.01, 0,1, 1]

### Functions

In [3]:
# def drop_nan_from_specific_columns (df,columns_to_check):
#     """
#     Drops all rows with nan values in specific columns in a dataframe
#     """
#     df.dropna(
#         axis=0,
#         how='any',
#         thresh=None,
#         subset=columns_to_check,
#         inplace=True
#     )
    
# def split_clf_and_params(best_estimator_clf):
#     """
#     Takes best estimator[clf] and outputs a list with clf and parameters
#     """
#     clf_and_params = str(best_estimator_clf)
#     clf_and_params = clf_and_params.replace(")", "")
#     clf_and_params_split = clf_and_params.split("(")
#     return clf_and_params_split

# def rmse_from_neg_mean_squared_error(neg_mean_squared_error):
#     """
#     Calculates RMSE from the neq mean squared error
#     """
#     rmse = np.sqrt(-(neg_mean_squared_error))
#     return (rmse)

# # functie maken om op basis van de grid search best estimator, het beste RMSE model te selecteren 
# def rmse_from_gridsearch_best_estimator(grid_search):
#     """
#     Calculates RMSE from the grid search best estimator
#     """
#     rmse = np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test)))
#     return (rmse)

# Get data
This step will load and combine several tables from CBS statline. 

Note: This step takes a number of minutes and without changes to the settings will give the same result. Therefor this code is commented out and the original dataset is loaded. 

In [4]:
%%time 
# ## CREATE NEW DATASET
# df_get_data_WMO= get_data(save_all=True)

# ## HARDCODED
datapath = '../../data/'
filename = 'df_get_data_WMO_WIJK_HUISHOUDENS_BEVOLKING_HEFFING_202104042111.parquet.gzip'
df_get_data_WMO = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_WMO_', datapath=datapath, filetype='parquet')

print(f"The shape of the dataframe from step 'Get Data': {df_get_data_WMO.shape}")
df_get_data_WMO.sample(5)

The shape of the dataframe from step 'Get Data': (936, 470)
CPU times: user 152 ms, sys: 56.1 ms, total: 208 ms
Wall time: 194 ms


,,perioden,typemaatwerkarrangement,wmoclienten,wmoclientenper1000inwoners,aantalinkomensontvangers,aantalinwoners,actieven1575jaar,afstandtotgrotesupermarkt,afstandtothuisartsenpraktijk,afstandtotkinderdagverblijf,afstandtotschool,alandbouwbosbouwenvisserij,bedrijfsmotorvoertuigen,bedrijfsvestigingentotaal,bevolkingsdichtheid,bfnijverheidenenergie,bouwjaarvanaf2000,bouwjaarvoor2000,dekkingspercentage,eenpersoonshuishoudens,eigendomonbekend,geboorterelatief,geboortetotaal,gehuwd,gemeentenaam,gemelectriciteitsverbruikappartement,gemelectriciteitsverbruikeigenwoning,gemelectriciteitsverbruikhoekwoning,gemelectriciteitsverbruikhuurwoning,gemelectriciteitsverbruiktussenwoning,gemelectriciteitsverbruiktweeondereenkapwoning,gemelectriciteitsverbruikvrijstaandewoning,gemgasverbruikappartement,gemgasverbruikeigenwoning,gemgasverbruikhoekwoning,gemgasverbruikhuurwoning,gemgasverbruiktussenwoning,gemgasverbruiktweeondereenkapwoning,gemgasverbruikvrijstaandewoning,gemgestandaardiseerdinkomenvanhuish,gemiddeldaardgasverbruiktotaal,gemiddeldehuishoudensgrootte,gemiddeldelektriciteitsverbruiktotaal,gemiddeldewoningwaarde,gemiddeldinkomenperinkomensontvanger,gemiddeldinkomenperinwoner,gescheiden,geweldsenseksuelemisdrijven,gihandelenhoreca,hjvervoerinformatieencommunicatie,huishonderofrondsociaalminimum,huishoudensmeteenlaaginkomen,huishoudensmetkinderen,huishoudenstot110vansociaalminimum,huishoudenstot120vansociaalminimum,huishoudenstotaal,huishoudenszonderkinderen,huurwoningentotaal,inbezitoverigeverhuurders,inbezitwoningcorporatie,jongerenmetjeugdzorginnatura,k0tot15jaar,k15tot25jaar,k20huishoudensmethoogsteinkomen,k20personenmethoogsteinkomen,k25tot45jaar,k40huishoudensmetlaagsteinkomen,k40personenmetlaagsteinkomen,k45tot65jaar,k65jaarofouder,klfinancieledienstenonroerendgoed,koopwoningen,mannen,marokko,matevanstedelijkheid,mediaanvermogenvanparticulierehuish,meestvoorkomendepostcode,mnzakelijkedienstverlening,motorfietsen,nederlandseantillenenaruba,nettoarbeidsparticipatie,nietwesterstotaal,omgevingsadressendichtheid,ongehuwd,opleidingsniveauhoog,opleidingsniveaulaag,opleidingsniveaumiddelbaar,oppervlakteland,oppervlaktetotaal,oppervlaktewater,overignietwesters,percentagebewoond,percentageeengezinswoning,percentagejongerenmetjeugdzorg,percentagemeergezinswoning,percentageonbewoond,percentagewerknemers,percentagewoningenmetstadsverwarming,percentagezelfstandigen,personenautos6jaarenouder,personenautosbrandstofbenzine,personenautosjongerdan6jaar,personenautosnaaroppervlakte,personenautosoverigebrandstof,personenautosperhuishouden,personenautostotaal,personenpersoortuitkeringao,personenpersoortuitkeringaow,personenpersoortuitkeringbijstand,personenpersoortuitkeringww,rucultuurrecreatieoverigediensten,scholenbinnen3km,sterfterelatief,sterftetotaal,suriname,totaaldiefstaluitwoningschuured,turkije,vernielingmisdrijftegenopenbareorde,verweduwd,vrouwen,westerstotaal,woningvoorraad,alleenstaande_mannen,alleenstaande_totaal_mannen_en_vrouwen,alleenstaande_vrouwen,ouder_in_eenouderhuishouden_mannen,ouder_in_eenouderhuishouden_totaal_mannen_en_vrouwen,ouder_in_eenouderhuishouden_vrouwen,overig_lid_huishouden_mannen,overig_lid_huishouden_totaal_mannen_en_vrouwen,overig_lid_huishouden_vrouwen,"partner_in_paar,_geen_kind(eren)_mannen","partner_in_paar,_geen_kind(eren)_totaal_mannen_en_vrouwen","partner_in_paar,_geen_kind(eren)_vrouwen",partner_in_paar_met_kind(eren)_mannen,partner_in_paar_met_kind(eren)_totaal_mannen_en_vrouwen,partner_in_paar_met_kind(eren)_vrouwen,persoon_in_institutioneel_huishouden_mannen,persoon_in_institutioneel_huishouden_totaal_mannen_en_vrouwen,persoon_in_institutioneel_huishouden_vrouwen,persoon_in_particulier_huishouden_mannen,persoon_in_particulier_huishouden_totaal_mannen_en_vrouwen,persoon_in_particulier_huishouden_vrouwen,thuiswonend_kind_mannen,thuiswonend_kind_totaal_mannen_en_vrouwen,thuiswonend_kind_vrouwen,totaal_personen_mannen,totaal_personen_totaal_mannen_en_vrouwen,totaal_personen_vrouwen,popaantalbasisonderwijs

# Preprocess --> Create test sets
This step will transform (select columns, impute, scale) the dataframe to be used in train/predict. 

In [5]:
%%time 
df_preprocessed = preprocess_data(df=df_get_data_WMO, save_all=True, personal_note="test123")

# print(f"The shape of the dataframe from step 'Preprocess': {df_preprocessed.shape}")
df_preprocessed.sample(5)

CPU times: user 3.13 s, sys: 38.1 ms, total: 3.17 s
Wall time: 3.17 s


,,wmoclienten,wmoclientenper1000inwoners,relative_huishoudensmetkinderen,relative_poparbeidsongeschiktheidtotaal,relative_ongehuwd,relative_poptotaleoppervlakte,relative_huishoudenszonderkinderen,relative_ouder_in_eenouderhuishouden_vrouwen,relative_alleenstaande_mannen,relative_popafstandtothuisartsenpraktijk,relative_popbevolkingsdichtheid
codering_regio,interval,,,,,,,,,,,
GM0317,2018,140.000000000000000,15.000000000000000,0.401774067310201,0.030728709394205,0.444359086918349,0.003698419666374,0.311766240542656,0.263388937664618,0.532265144863916,0.000109745390694,0.032265144863916
GM1930,2018,2025.000000000000000,24.000000000000000,0.357555567123002,0.041492794912109,0.439967844138649,0.000990507488799,0.306517099578367,0.351054523100203,0.665756372062231,0.000008274916364,0.013594505455534
GM0342,2017,1005.000000000000000,22.000000000000000,0.349910749702499,0.040763831364171,0.435758817630902,0.001012120155208,0.311483538278461,0.086650390199241,0.227143915943672,0.000017439072241,0.021624449579282
GM0302,2017,505.000000000000000,19.000000000000000,0.393006859756098,0.039788784768704,0.415402350141306,0.004816674103823,0.325552591463415,0.039045069165551,0.113416629480887,0.000059497248252,0.007771828052953
GM0613,2019,480.000000000000000,19.000000000000000,0.434381139489194,0.029282576866764,0.451030825847810,0.000940208143722,0.304223968565815,0.169364093229393,0.313996280321317,0.000035613944838,0.046100273040244


In [6]:
# df_preprocessed

In [7]:
# param_grid=[]

In [8]:
# if len(param_grid)==0:
#     print("fds")

In [9]:
# # For testing train
# datapath = '../../data/'
# # filename = 'df_preprocessed_202104042151_All.parquet.gzip'
# filename = 'df_preprocessed_202104042151_Boerenverstand_Maikel.parquet.gzip'
# # filename = 'df_preprocessed_202104042151_Minimum_Maikel.parquet.gzip'
# # filename = 'df_preprocessed_202104042151_No_Relative.parquet.gzip'
# df_preprocessed = pd.read_parquet(datapath + filename)
# input_filename = filename

In [10]:
# df_preprocessed.head()

In [11]:
# df = df_preprocessed.copy()

# Train

## Train and fit multiple models

In [12]:
gridsearch_object = train_and_fit_models(df_preprocessed=df_preprocessed,
                                        filename_input="test_123",
                                        save_all=True,
                                        personal_note="test123")

The model with the best train score is:
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
This model has a train score (RMSE) of: 5.438133123097392
This model has a test score (RMSE) of: 5.439500768211023


## Use best trained model from gridsearch

In [14]:
best_trained_model = gridsearch_object.best_estimator_